# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

In [3]:
#Import Packages
import re
import Bio.SeqIO

In [4]:
#Read in sequencing data(from Lecture 9)
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

In [5]:
#Convert SeqRecords objects to a list of strings(Lecture 9)
seqreads_str = []
for seqrecord in seqreads:
    sequence = str(seqrecord.seq) # convert sequence part to string
    seqreads_str.append(sequence) # add string sequence to list

In [34]:
#Converst list of sequences to dataframe 
import pandas as pd
import numpy as np

df = pd.DataFrame (seqreads_str, columns = ['Sequence'])

print(df)

                                               Sequence
0     GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...
1     CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...
2     CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...
3     TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...
4     ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...
...                                                 ...
9995  GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...
9996  GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...
9997  TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...
9998  GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...
9999  CGTAGGATTGAATTAGGCGGCCGCCTATGGTGCACTATTATTTATC...

[10000 rows x 1 columns]


In [23]:
#Make a function to get R.C from each strong(Lecture 9)
def reverse_complement(seq, unk_partner = 'N'):
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'} #Make Dictionary of basepairs
    rseq = '' #Make Empty String
    for a in seq: #loops over bases in sequence
        if a in base_partner:
            # look up the complementary base in the dictionary
            pair = base_partner[a]
            rseq = pair + rseq #build reverse comp string
        else:
            rseq = unk_partner + rseq
    return rseq 

In [45]:
#Add a column of the Reverse compliment for each sequence 
df['Reverse_Comp'] = df['Sequence'].apply(reverse_complement)
print(df)

                                               Sequence  \
0     GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
1     CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...   
2     CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...   
3     TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...   
4     ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...   
...                                                 ...   
9995  GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...   
9996  GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...   
9997  TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
9998  GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...   
9999  CGTAGGATTGAATTAGGCGGCCGCCTATGGTGCACTATTATTTATC...   

                                           Reverse_Comp  
0     TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...  
1     TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...  
2     TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...  
3     TCAGATGAATGGTAGTTGGTGATAGCATGAGGTTGGGTCGGATGGT...  
4

In [48]:
#Make Function to Determine if sequences are NA 
#NAend='CACGATAGATAAATAATAGTGCACCAT'

def read_typeNA(seqreads):
    
    # compile the type search pattern
    type_pattern = re.compile("(CACGATAGATAAATAATAGTGCACCAT)") #need to change

    match = type_pattern.search(seqreads)
    #1 = true and 0 equals false
    if match:
        return 1
    else:
        return 0

In [49]:
#Add a column to evalulate if the Sequence is the NA virus strain
df['NA?'] = df['Reverse_Comp'].apply(read_typeNA)
print(df)



                                               Sequence  \
0     GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
1     CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...   
2     CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...   
3     TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...   
4     ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...   
...                                                 ...   
9995  GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...   
9996  GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...   
9997  TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
9998  GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...   
9999  CGTAGGATTGAATTAGGCGGCCGCCTATGGTGCACTATTATTTATC...   

                                           Reverse_Comp  NA?  HA?  
0     TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...    1    0  
1     TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...    0    1  
2     TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...    1    0  
3     TCAGATGAATGGT

In [50]:
#Make Function to Determine if sequences are HA
#HAend='CCGGATTTGCATATAATGATGCACCAT'

def read_typeHA(seqreads):
    
    # compile the type search pattern
    type_pattern = re.compile("(CCGGATTTGCATATAATGATGCACCAT)") #need to change

    match = type_pattern.search(seqreads)
    #1 = true and 0 equals false
    if match:
        return 1
    else:
        return 0

In [51]:
#Add a column to evalulate if the Sequence is the NA virus strain
df['HA?'] = df['Reverse_Comp'].apply(read_typeHA)
print(df)

                                               Sequence  \
0     GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
1     CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...   
2     CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...   
3     TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...   
4     ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...   
...                                                 ...   
9995  GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...   
9996  GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...   
9997  TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
9998  GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...   
9999  CGTAGGATTGAATTAGGCGGCCGCCTATGGTGCACTATTATTTATC...   

                                           Reverse_Comp  NA?  HA?  
0     TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...    1    0  
1     TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...    0    1  
2     TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...    1    0  
3     TCAGATGAATGGT

In [31]:
#Make function to read barcodes(lecture 9)
def read_barcode(seqread, bclen=16, upstream='AGGCGGCCGC'):
    # compile the barcode search pattern
    barcode_pattern = re.compile(upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # search for the barcode pattern
    match = barcode_pattern.search(seqread)
    
    if match:
        barcode = match.group('barcode')
        return barcode
    else:
        return None

In [52]:
df['Barcode'] = df['Reverse_Comp'].apply(read_barcode)
print(df)

                                               Sequence  \
0     GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
1     CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...   
2     CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...   
3     TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...   
4     ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...   
...                                                 ...   
9995  GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...   
9996  GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...   
9997  TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...   
9998  GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...   
9999  CGTAGGATTGAATTAGGCGGCCGCCTATGGTGCACTATTATTTATC...   

                                           Reverse_Comp  NA?  HA?  \
0     TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...    1    0   
1     TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...    0    1   
2     TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...    1    0   
3     TCAGATGAA

In [58]:
print(df["NA?"].mean())
print(df["HA?"].mean())

0.4122
0.5409


1. How many reads map to HA, and how many reads map to NA?
41.22 percent of the sequences are NA strain and 54.09 are HA strain with 4.69 percent being neither NA or HA.

2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [ ]:
def get_monthly_mean(df):

    df_grouped = df.groupby('Month_of_Year')['CallsPresented'].mean()

    #Then you can pass the column to a list or just return the grouped df, 
    #whatever suits your use case better

    return df_grouped